In [ ]:
import argparse
import collections
import torch
import numpy as np
import model.loss as module_loss
import model.metric as module_metric
from parse_config import ConfigParser
from utils.util import create_model, create_dataloader, create_trainer


In [ ]:
# fix random seeds for reproducibility
SEED = 125
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

In [ ]:
args = argparse.ArgumentParser(description='Emotion Reasoning in Daily Life')
args.add_argument('-c', '--config', default='train.json', type=str,
                  help='config file path (default: None)')
args.add_argument('-r', '--resume', default=None, type=str,
                  help='path to latest checkpoint (default: None)')
args.add_argument('-d', '--device', default="0", type=str,
                  help='indices of GPUs to enable (default: all)')

# custom cli options to modify configuration from default values given in json file.
CustomArgs = collections.namedtuple('CustomArgs', 'flags type target')
options = [
    CustomArgs(['--lr', '--learning_rate'], type=float, target='optimizer;args;lr'),
    CustomArgs(['--bs', '--batch_size'], type=int, target='data_loader;args;batch_size')
]

In [ ]:
config = ConfigParser.from_args(args, options)

In [ ]:
config._config['name'] += "_add_text"

In [ ]:
print(config._config)

In [ ]:
logger = config.get_logger('train')

In [ ]:
from data_loader_add.data_loader_add import AddMEmoRDataLoader

In [ ]:
# setup data_loader instances
data_loader = AddMEmoRDataLoader(config)
valid_data_loader = data_loader.split_validation()

In [ ]:
from model_add.model_add import AddAMER

In [ ]:
# build model architecture, then print to console
model = AddAMER(config)

In [ ]:
# get function handles of loss and metrics
criterion = getattr(module_loss, config['loss'])
metrics = [getattr(module_metric, met) for met in config['metrics']]

In [ ]:
from trainer_add import AddMEmoRTrainer

In [ ]:
trainer = AddMEmoRTrainer(model, criterion, metrics,
                      config=config,
                      data_loader=data_loader,
                      valid_data_loader=valid_data_loader,
                    )      

In [ ]:
print(next(trainer.model.parameters()).is_cuda)

In [ ]:
trainer.train()

In [ ]:
exit()

In [ ]:
target, U_v, U_a, U_t, U_p, M_v, M_a, M_t, target_loc, umask, seg_len, n_c, U_t_add = next(iter(data_loader))

In [ ]:
U_t.shape

In [ ]:
U_t

In [ ]:
U_t_add.shape

In [ ]:
import torch.nn as nn

In [ ]:
enc_t_add = nn.Sequential(
            nn.Linear(24, 128 * 4),
            nn.ReLU(),
            nn.Linear(128 * 4, int(0.5 * 128)), # D_e = 128
        )    